In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from itertools import permutations

In [ ]:
A = [
    [3, 2, 1, 1, 0, 0, 0],
    [6, 1, 5, 0, 1, 0, 0],
    [4, 1, 5, 0, 0, 1, 0],
    [3, 1, 4, 0, 0, 0, 1],
]
A = np.array(A)

b = [60, 120, 100, 80]
b = np.array(b)

c = [30, 20, 15, 0, 0, 0, 0]
c = np.array(c)

m = 4 # Number of inequalities (except x >= 0)
n = 7 # Number of variables

In [ ]:
def get_bases(A, m, n):
    '''Eine zulässige Basis entspricht genau einem Eckpunkt. Mehrere basen können den selben Eckpunkt repräsentieren'''
    rank = m
    columns = n
    indexes_list = list(permutations(range(columns), int(rank)))    
    for indexes in indexes_list:
        unselected_indices = [i for i in range(columns) if i not in indexes]
        basis = A[:, indexes]
        n = A[:, unselected_indices]
        yield basis, n, list(indexes), list(unselected_indices)

In [ ]:
def select_base_manually(A, selected_indexes):
    B = A[:, selected_indexes]
    unselected_indices = [i for i in range(A.shape[1]) if i not in selected_indexes]
    N = A[:,unselected_indices]
    return B, N, selected_indexes, unselected_indices

In [ ]:
def get_basis_solution(B, b):
    B_inverse = np.linalg.inv(B)
    return B_inverse @ b, B_inverse

In [ ]:
#B , N, x_indexes, n_indexes = get_bases(A, m, n).__next__()

# Alternatively, use this:
B, N, x_indexes, n_indexes = select_base_manually(A, [3, 4, 5, 6])

In [ ]:
x_B, b_inverted = get_basis_solution(B, b)

In [ ]:
def is_basis_solution_valid(basis_solution):
    return np.all(basis_solution >= 0)

In [ ]:
is_basis_solution_valid(x_B)

In [ ]:
def get_reduced_costs(c, x_indexes, n_indexes, n, b_inverted):
    c_B = []
    for i in x_indexes:
        c_B.append(c[i])
    c_B = np.array(c_B)
    #c_B = c[x_indexes]
    
    c_n = []
    for i in n_indexes:
        c_n.append(c[i])
    c_n = np.array(c_n)
    #c_n = c[n_indexes]
    
    return c_n - c_B  @ b_inverted @ n

reduced_costs = get_reduced_costs(c, x_indexes, n_indexes, N, b_inverted)

In [ ]:
def get_alpha(b_inverted, n):
    return b_inverted @ n
alphas = get_alpha(b_inverted, N)

In [ ]:
def is_optimal(reduced_costs):
    return np.all(reduced_costs <= 0)

In [ ]:
def has_infinite_optimal_solution(reduced_costs, alphas, m, n):
    # This somehow doesn't work...
    for j in range(0, n-m):
        if reduced_costs[j] > 0:
            all_alphas_ok = True
            for alpha in alphas[j]:
                if alpha > 0:
                    all_alphas_ok = False
                    break
            if all_alphas_ok:
                return True
    return False

In [ ]:
def find_better_neighbor(reduced_costs, A, B, X_b, x_indexes, n_indexes, n, m):
    basis_index_to_add = None # this is j
    for j in range(n-m):
        reduced_cost = reduced_costs[j]
        if reduced_cost > 0:
            basis_index_to_add = j
    if basis_index_to_add is None:
        return None # The solution is already optimal
    
    # Add column to basis
    x_indexes.append(n_indexes[basis_index_to_add])
    column_to_add = A[:, n_indexes[basis_index_to_add]] 
    B_extended = np.column_stack((B, column_to_add))
    
    b_extended_inv = np.linalg.inv(B) # Not sure if this should rather be B_extended

    N = A[:, n_indexes]
    n_indexes.remove(n_indexes[basis_index_to_add])
    #print(N.shape, n_indexes)
    extended_alphas = get_alpha(b_extended_inv, N)
    relevant_alphas = extended_alphas[:, basis_index_to_add]
    alphas_bigger_zero = relevant_alphas > 0
    betas_over_alphas = [X_b[i] / relevant_alphas[i] for i in range(X_b.shape[0])]
    
    huge_penalties = (1-alphas_bigger_zero) * 1e9
    betas_over_alphas += huge_penalties
    i = np.argmin(betas_over_alphas)

    # Remove column i from B_extended
    B_new = np.delete(B_extended, i, 1)
    index_to_append_to_n_indexes = x_indexes.pop(i)
    n_indexes.append(index_to_append_to_n_indexes)
    return B_new, x_indexes, n_indexes
    

In [ ]:
next_basis, new_indices, new_n_indices = find_better_neighbor(reduced_costs, A, B, x_B, x_indexes, n_indexes, n, m)

In [ ]:
B = next_basis
x_indexes =  new_indices
n_indexes = new_n_indices

In [ ]:
B